In [15]:
from graphframes import GraphFrame

In [88]:
inputPathVertexes = "data/Ex54/data/vertexes.csv"
inputPathEdges = "data/Ex54/data/edges.csv"
outputPath = "resOut_ex54/"

In [89]:
# Read the content of vertexes.csv
vDF = spark.read.load(inputPathVertexes,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [90]:
#vDF.printSchema()
#vDF.show()

In [91]:
# Read the content of edges.csv
eDF = spark.read.load(inputPathEdges,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [92]:
#eDF.printSchema()
#eDF.show()

In [93]:
# Select only links of type friend
filteredEdges = eDF.filter("linktype='friend' ")

In [95]:
# Create the input graph
g = GraphFrame(vDF, filteredEdges)

In [102]:
# Select the paths/motifs (v1)-[]->(v2);!(v2)-[]->(v1)
selectedPaths = g.find("(userx)-[]->(usery);!(usery)-[]->(userx)")

In [103]:
selectedPaths.printSchema()
selectedPaths.show()

root
 |-- userx: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- age: integer (nullable = true)
 |-- usery: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- age: integer (nullable = true)

+---------------+---------------+
|          userx|          usery|
+---------------+---------------+
|[u4, David, 29]|[u1, Alice, 34]|
|[u1, Alice, 34]|  [u2, Bob, 36]|
+---------------+---------------+



In [113]:
# Select only the ids of the users (rename the selected columns)
selectedPairsDF=selectedPaths.selectExpr("userx.id as IdFriend","usery.id as IdNotFriend")

In [116]:
selectedPairsDF.printSchema()
selectedPairsDF.show()

root
 |-- IdFriend: string (nullable = true)
 |-- IdNotFriend: string (nullable = true)

+--------+-----------+
|IdFriend|IdNotFriend|
+--------+-----------+
|      u4|         u1|
|      u1|         u2|
+--------+-----------+



In [117]:
# Save the result in the output folder
selectedPairsDF.write.csv(outputPath, header=True)